### Preprocessing images data
1. rgb2gray
2. HistogramMatching

In [1]:
import SimpleITK as sitk

images_location = '../Data/FIRE/Images/'
output_location = '../Data/FIRE/Results/'

import os
os.makedirs(output_location, exist_ok=True)

# use 'P' images: total 49 image pairs
data_categories = ['P']
# data_numbers = range(49)  # when using all images
data_numbers = range(1)

for cat in range(len(data_categories)):
    for num in data_numbers:
        # make dir for each pair
        iter_pair_name = data_categories[cat] + "%02d" % (num+1)
        iter_output_location = output_location + iter_pair_name
        os.makedirs(iter_output_location, exist_ok=True)
        
        print(iter_pair_name)
        # fixed image: #1 image in pair
        gray_fixed = sitk.ReadImage(images_location + f'{iter_pair_name}_1.jpg', sitk.sitkUInt8)
        gray_fixed.SetSpacing([1,1])
        # moving image: #2 image in pair
        gray_moving = sitk.ReadImage(images_location + f'{iter_pair_name}_2.jpg', sitk.sitkUInt8)
        gray_moving.SetSpacing([1,1])
        
        # HistogramMatching
        matcher = sitk.HistogramMatchingImageFilter()
        matcher.SetNumberOfHistogramLevels(128)
        matcher.SetNumberOfMatchPoints(7)
        matcher.ThresholdAtMeanIntensityOn()
        histogram_moving = matcher.Execute(gray_moving, gray_fixed)
        
        sitk.WriteImage(gray_fixed, iter_output_location + f'/{iter_pair_name}_fixed.mha', True)
        sitk.WriteImage(histogram_moving, iter_output_location + f'/{iter_pair_name}_moving.mha', True)        

P01


### Registration

In [2]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.addpath(r'..\CCM', nargout=0)
eng.addpath(r'..\CCM\util', nargout=0)

In [3]:

import SimpleITK as sitk
import sys
sys.path.append('..')
import os 

from CCM import framework as methods
from CCM import assessment as assess

points_location = '../Data/FIRE/Ground Truth/'
mask_location = '../Data/FIRE/Masks/'
output_location = '../Data/FIRE/Results/'

def GetPointsFromFireTxt(location, pair_name):
    file_name = location + f'control_points_{pair_name}_1_2.txt'
    file = open(file_name)
    
    pfs = []; pms = []
    for line in file.readlines():
        split_line = line.split(' ')
        pf = [split_line[0], split_line[1]]
        pm = [split_line[2], split_line[3]]

        pfs.append(pf)
        pms.append(pm)
        
    return pfs, pms

data_categories = ['P']
# data_numbers = [range(49)]  # when using all images
data_numbers = [range(1)]

mask = sitk.ReadImage(mask_location + 'mask.png', sitk.sitkUInt8) > 1
feat_mask = sitk.ReadImage(mask_location + 'feature_mask.png', sitk.sitkUInt8) > 1
feat_mask = sitk.BinaryErode(feat_mask, [100,100])

for cat in range(len(data_categories)):
    for num in data_numbers[cat]:
        
        # your turn
        iter_pair_name = data_categories[cat] + "%02d" % (num+1)
        iter_output_location = output_location + iter_pair_name
        print(iter_pair_name)
        
        fixed = sitk.ReadImage(iter_output_location + f'/{iter_pair_name}_fixed.mha')
        moving = sitk.ReadImage(iter_output_location + f'/{iter_pair_name}_moving.mha')
        fixed_points, moving_points = GetPointsFromFireTxt(points_location, iter_pair_name)

        # CCM
        print('\rCCM ...', end='')
        reg = methods.Registration(fixed=fixed, moving=moving, matlab_eng=eng, fixed_mask=feat_mask, moving_mask=feat_mask)
        reg.PerformCCMRegistration(
            nlevel=[4], nmigr=[3], niter=[200], step=1, sigma=16,
            feature_detector=reg.detectorFAST, feature_detector_size=7, feature_detector_angle=150, feature_detector_ratio=[4], feature_detector_num=[2000],
            feature_descriptor=reg.descriptorBRISK, feature_descriptor_radius=[0, 3, 5, 7, 9, 11], feature_descriptor_samplenum=[1, 10, 15, 20, 25, 30], feature_descriptor_dmax=7, feature_descriptor_dmin=999,
            feature_match_ratio=1, feature_match_crosscheck=True,
            route_merge=True, route_check=True, route_stay=0.01,
            ccm_k=[50], ccm_e=[0.1], ccm_j=[0.5], ccm_t=[0.5], ccm_z=[0.5])
        print('\rCCM, done.')
        reg.OutputRegistrationResult(iter_output_location + f'/{iter_pair_name}_ccm_')
        print(assess.Assessment(registration=reg, mask=mask, fixed_points=fixed_points, moving_points=moving_points).AssessRegistration()) 

c:\Users\Administrator\Documents\Envs\crowd\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


P01
CCM, done.
{'mse': [95.4135211090274, 74.79093350419375, 5813887], 'jac': [0.9713647933380106, 0.11934576370956784, 5813887], 'tre': [3.656571319332495, 2.5651593172765286, 10], 'err': [-1, -1, -1], 'SDlogJ': 0.4460127354608442, 'dice': [], 'hdd': [], 'time': [5.981007814407349]}
